In [1]:
! pip install arabic-reshaper python-bidi


In [2]:
import pandas as pd
import numpy as np
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
import arabic_reshaper
from bidi.algorithm import get_display



nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')

C:\Users\Campos\AppData\Local\Temp\ipykernel_19008\4045042623.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Campos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data Preprocessing

In [3]:
positive_data_train = pd.read_csv('../Data/train_Arabic_tweets_positive_20190413.tsv', sep='\t', header=None, names=['tweet'])
negative_data_train = pd.read_csv('../Data/train_Arabic_tweets_negative_20190413.tsv', sep='\t', header=None, names=['tweet'])
positive_data_test = pd.read_csv('../Data/test_Arabic_tweets_positive_20190413.tsv', sep='\t', header=None, names=['tweet'])
negative_data_test = pd.read_csv('../Data/test_Arabic_tweets_positive_20190413.tsv', sep='\t', header=None, names=['tweet'])

In [4]:
# Initialize stop words and stemmer for Arabic
stop_word = set(stopwords.words('arabic'))
stemmer = ISRIStemmer()

def preproces(text):
    text  = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    token = nltk.word_tokenize(text)
    word =[stemmer.stem(word) for word in token if word not in stop_word]
    return ' '.join(word)


In [5]:
positive_data_train['processed_tweet'] = positive_data_train['tweet'].apply(preproces)
positive_data_train


,tweet,processed_tweet
pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,تحل نود نقل دعء له، بحث فين قوة، انن مكسورون، قوة
pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,وفي نهي قىٰ معك حدإل رأىٰ جمل روح اماالمنبهر مظا
pos,من الخير نفسه 💛,خير نفس
pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,زلزلالملعبنصرنابيلعب علي همه رضى بغر قمه جرد س...
pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,شيء وحد وصل علم سير ترى كان شجع درد ضد نصر
...,...,...
pos,السحب الليلة على الايفون .. رتويت للمرفقة وطبق...,سحب ليل ايف رتي رفق طبق شرط
pos,😂 لابسة احمر ليه يا ست انتي ايه المناسبة 😂,لبس حمر ليه انت ايه نسب
pos,كلاام جمييل تستاهل(من احبه الله جعل محبته ف قل...,كلام جمييل تستاهلمن احب الل حبت قلب بشر
pos,- ألطف صورة ممكن تعبر عن رمضان 💙,لطف صور مكن عبر رمض


In [6]:
negative_data_train['processed_tweet'] = negative_data_train['tweet'].apply(preproces)
positive_data_test['processed_tweet'] = negative_data_test['tweet'].apply(preproces)
negative_data_test['processed_tweet'] = negative_data_test['tweet'].apply(preproces)


In [7]:
positive_data_train['label'] = 1
negative_data_train['label'] = 0
positive_data_test['label'] = 1
negative_data_test['label'] = 0

In [8]:
train_data = pd.concat([positive_data_train, negative_data_train], ignore_index=True)
train_data


,tweet,processed_tweet,label
0,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...,تحل نود نقل دعء له، بحث فين قوة، انن مكسورون، قوة,1
1,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...,وفي نهي قىٰ معك حدإل رأىٰ جمل روح اماالمنبهر مظا,1
2,من الخير نفسه 💛,خير نفس,1
3,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...,زلزلالملعبنصرنابيلعب علي همه رضى بغر قمه جرد س...,1
4,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...,شيء وحد وصل علم سير ترى كان شجع درد ضد نصر,1
...,...,...,...
45270,كيف ترى أورانوس لو كان يقع مكان القمر ؟ 💙💙 كوك...,ترى رنس يقع كان قمر ؟ كوكب شمس بلغ قطر كرة جوف...,0
45271,احسدك على الايم 💔,حسد ايم,0
45272,لأول مرة ما بنكون سوا 💔,لأل مرة بنك سوا,0
45273,بقله ليش يا واطي 🤔,بقل ليش وطي,0


In [9]:
test_data = pd.concat([positive_data_test, negative_data_test], ignore_index=True)
test_data

,tweet,processed_tweet,label
0,#الهلال_الاهلي فوز هلالي مهم الحمد لله 💙 زوران...,هلالالاهلي فوز هلل مهم حمد لله زور سلم برا بدل...,1
1,صباحك خيرات ومسرات 🌸,صبح خير مسر,1
2,#تأمل قال الله ﷻ :- _*​﴿بواد غير ذي زرع ﴾*_ 💫💫...,أمل قال الل بود زرع ومع هتف دعء رزق ثمر مهماكا...,1
3,😂😂 يا جدعان الرجاله اللي فوق ال دول خطر ع تويت...,جدع رجل الل ال دول خطر يتر ورب مش سلب يوم دخل ...,1
4,رساله صباحيه : 💛 اللهم اسألك التوفيق في جميع ا...,رسل صبح لهم سأل وفق امر كتب ردس ولد جمع وتى سل...,1
...,...,...,...
11499,ربي اغفر لي و لوالدي و لأحبتي و للمؤمنين و الم...,ربي غفر للد أحب ؤمن ؤمن سلم سلم حيء نهم امو,0
11500,ربي يسعدنا وياكم 💛,ربي سعد ويا,0
11501,يتحدثون عن اخلاق حسين ونجوم فرقهم نهاياتهم الر...,حدث خلق حسن نجم فرق نهي ريض ليم خجل ختلف تفق ح...,0
11502,صباحكم احتفالية لم تكتمل، وصاحب الاحتفاليه ماك...,صبح احتفالية كتمل، صحب احتفاليه اكمل برا برضوه...,0


# Feature Extraction

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vector = TfidfVectorizer(max_features=5000)
x_train = vector.fit_transform(train_data['processed_tweet']).toarray()
y_train = train_data['label'].values

x_test = vector.transform(test_data['processed_tweet']).toarray()
y_test = test_data['label'].values

In [11]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((45275, 5000), (45275,), (11504, 5000), (11504,))

# Build MOdels


### a. Traditional Machine Learning Alg

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

RF_model = RandomForestClassifier(n_estimators=100, random_state=42)
RF_model.fit(x_train, y_train)

In [ ]:
y_pred = RF_model.predict(x_test)
acc = accuracy_score(y_test, y_pred)
acc

In [ ]:
class_report = classification_report(y_test, y_pred,output_dict=True)
report = pd.DataFrame(class_report).transpose()
report

### b.	Deep learning technique (RNN, LSTM)

In [ ]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
tokanizer = Tokenizer(num_words=5000)
tokanizer.fit_on_texts(train_data['processed_tweet'])

x_train_seq = tokanizer.texts_to_sequences(x_train['processed_tweet'])
x_test_seq = tokenizer.texts_to_sequences(test_data['processed_tweet'])
x_valid_seq = tokanizer.texts_to_sequences(x_valid['processed_tweet'])

x_train_pad = pad_sequences(x_train_seq, maxlen=200)
x_test_pad = pad_sequences(x_test_seq, maxlen=200)
x_valid_pad = pad_sequences(x_valid_seq, maxlen=200)

In [ ]:
model = sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2))
model.add(Dense(1,activation='sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy',optimalizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X_train_pad, y_train, epoch=100, batch_size=32,validation_data+(x_valid_pad, y_valid) )

In [ ]:
accuracy = model.evalute(x_test_pad,y_test)
print('Accuracy:' accuracy )

### c.	Transformers using HuggingFace

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

In [ ]:
train_encodings = encode_reviews(train_data['processed_tweet'].tolist())
test_encodings = encode_reviews(test_data['processed_tweet'].tolist())

In [ ]:
train_labels = tf.convert_to_tensor(y_train)
test_labels = tf.convert_to_tensor(y_test)

In [ ]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

In [ ]:
model.fit(train_encodings['input_ids'], train_labels, epochs=2, batch_size=8, validation_data=(test_encodings['input_ids'], test_labels))


In [ ]:
accuracy = model.evaluate(test_encodings['input_ids'], test_labels)
accuracy